In [1]:
import torch

In [2]:
from model import *

2024-04-27 11:28:23 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
from torch.nn.parallel import DataParallel

In [4]:
# Initialize your model
model = Model(device=device)


/home/hparashar/mtech_projects/speech_understanding/SU-PA3/.venv/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [11]:
model = DataParallel(model)

In [12]:
# Define the path to the .pt file
file_path = "models/Best_LA_model_for_DF.pth"
dictModel = torch.load(file_path,map_location=device)
model.load_state_dict(dictModel)

In [13]:
dictModel.keys()

odict_keys(['module.pos_S', 'module.master1', 'module.master2', 'module.ssl_model.model.mask_emb', 'module.ssl_model.model.feature_extractor.conv_layers.0.0.weight', 'module.ssl_model.model.feature_extractor.conv_layers.0.0.bias', 'module.ssl_model.model.feature_extractor.conv_layers.0.2.1.weight', 'module.ssl_model.model.feature_extractor.conv_layers.0.2.1.bias', 'module.ssl_model.model.feature_extractor.conv_layers.1.0.weight', 'module.ssl_model.model.feature_extractor.conv_layers.1.0.bias', 'module.ssl_model.model.feature_extractor.conv_layers.1.2.1.weight', 'module.ssl_model.model.feature_extractor.conv_layers.1.2.1.bias', 'module.ssl_model.model.feature_extractor.conv_layers.2.0.weight', 'module.ssl_model.model.feature_extractor.conv_layers.2.0.bias', 'module.ssl_model.model.feature_extractor.conv_layers.2.2.1.weight', 'module.ssl_model.model.feature_extractor.conv_layers.2.2.1.bias', 'module.ssl_model.model.feature_extractor.conv_layers.3.0.weight', 'module.ssl_model.model.featur

In [14]:
model.load_state_dict(dictModel)

<All keys matched successfully>